In [1]:
from consensus.PBFT_Simulator import PBFT_Simulator
from consensus.QPBFT_Simulator import QPBFT_Simulator
from consensus.Proposed_Simulator import Proposed_Simulator
import time

In [2]:
consensus = PBFT_Simulator(10, 2)
start = time.time()
for i in range(100):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} sec")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 1, Faulty => False, Send_messages => ''                                 , Receive_messages => [('Hello world 0', 'request', -1)]
IdUser => 2, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 3, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 4, Faulty => True , Send_messages => ''                                 , Receive_messages => []
IdUser => 5, Faulty => True , Send_messages => ''                                 , Receive_messages => []
IdUser => 6, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 7, Faulty => False, Send_messages => ''                                 , Receive_messages => []
IdUser => 8, Faulty => False, Send_messages => ''                                 , Receive_messag

In [3]:
del consensus

In [4]:
consensus = QPBFT_Simulator(6, 4, 2)
start = time.time()
for i in range(100):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} s")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 0, Role =>  Manager, Faulty =>  False, Send_messages_log => ''                                 , Receive_messages_log => [('Hello world 0', 'request', -1)]
IdUser => 1, Role =>  Manager, Faulty =>   True, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 2, Role =>  Manager, Faulty =>  False, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 3, Role =>  Manager, Faulty =>   True, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 4, Role =>  Manager, Faulty =>  False, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 5, Role =>  Manager, Faulty =>  False, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 6, Role =>    Voter, Faulty =>  False, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 7, Rol

In [5]:
del consensus

In [6]:
consensus = Proposed_Simulator(6, 4, 5, 2)
start = time.time()
for i in range(100):
    consensus.send_request(f"Hello world {i}")
end = time.time()
print(f"Spend time => {end - start} s")
print(f"The number of success-proof => {consensus.success_proof} times")

Request Phase
IdUser => 3, Role =>  Master, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 9, Role =>   Slave, Send_messages_log => ''                                 , Receive_messages_log => [('Hello world 0', 'request', -1)]
IdUser => 1, Role =>  Master, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 5, Role =>  Master, Send_messages_log => ''                                 , Receive_messages_log => []
IdUser => 2, Role =>  Master, Send_messages_log => ''                                 , Receive_messages_log => []
------------------------------
Prepare Internal Phase
IdUser => 3, Role =>  Master, Send_messages_log => ''                                 , Receive_messages_log => [('Hello world 0', 'prepare', 9)]
IdUser => 9, Role =>   Slave, Send_messages_log => ('Hello world 0', 'prepare', 9)    , Receive_messages_log => [('Hello world 0', 'request', -1)]
IdUser => 1, Role =>  Master, Se

In [7]:
del consensus